In [7]:
from langgraph.graph import StateGraph,END,START
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict,Union
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage,AIMessage

load_dotenv()
llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash')

class Agentstate(TypedDict):
    message:list[Union[AIMessage,HumanMessage]]

In [3]:
def process(state:Agentstate)->Agentstate:
    """this is the node."""
    response = llm.invoke(state['message'])
    print("this is model response:",response.content)
    state['message'].append(AIMessage(content=response.content))
    print("current state message",state['message'])
    return state
    

In [4]:
graph = StateGraph(Agentstate)
graph.add_node("process",process)
graph.add_edge(START,"process")
graph.add_edge("process",END)
app = graph.compile()

In [6]:
history = []
user_input = input("Enter your Query:")
while user_input != "exit":
    history.append(HumanMessage(content=user_input))
    result = app.invoke({'message':history})
    history = result["message"]
    user_input = input("Enter: ")

with open("logger.txt","w") as f:
    for message in history:
        if isinstance(message,HumanMessage):
            f.write(f"You:{message.content}\n")
        elif isinstance(message,AIMessage):
            f.write(f"AI:{message.content}")
    f.write("End of the conversation.")

KeyboardInterrupt: 